# Alura - Imersão IA 3ª Edição
## Projeto agentes com API Google Gemini
### hvallebr - 17-05-2025

Objetivo: Busca uma receita culinária com base nos ingredientes que você tem disponível. A partir de uma lista de ingredientes, será exibida uma lista de receitas onde é possível utilizar estes ingredientes. Após escolher uma das receitas, você receberá as instruções completas desta receita, como lista total de ingredientes, modo de preparo, tempo de preparo, rendimento (porções).

In [93]:
%pip -q install google-genai google-adk

In [94]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [95]:
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
import re # Para extrair o nome da receita da string

warnings.filterwarnings("ignore")

In [96]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [97]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [98]:
# --- NOVO: Função para extrair o nome da receita ---
def extract_recipe_names(raw_output: str) -> list[str]:
    """
    Extrai os nomes das receitas de uma string formatada pelo agente_buscador.
    Retorna uma lista de strings com os nomes das receitas.
    """
    names = []
    # Expressão regular para encontrar "Número - Nome da Receita"
    # Captura tudo após " - " até o final da linha
    pattern = re.compile(r'^\d+\s*-\s*(.+)$', re.MULTILINE)
    for line in raw_output.splitlines():
        match = pattern.match(line.strip())
        if match:
            names.append(match.group(1).strip())
    return names

In [99]:
# --- AGENTE 1: BUSCADOR DE RECEITAS ---
def agente_buscador(topico: str) -> tuple[str, list[str]]:
    """
    Busca receitas com base nos ingredientes e retorna a lista formatada
    e uma lista com os nomes das receitas.
    """
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um instrutor de culinária.
        Sua tarefa é usar a ferramenta de busca do google (Google Search) para buscar receitas com base nos ingredientes fornecidos.
        Nesta etapa você só precisa se preocupar com o nome das receitas, que serão utilizadas para detalhamento em uma etapa seguinte.
        Cada receita deve ser exibida em uma linha, retorne uma lista com numeração sequencial das receitas no formato (1 - Bolo de Cenoura\n2 - Frango Xadrez\n).
        Exiba o número e nome de cada receita em linhas separadas.
        """,
        description="Agente buscador de receitas no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: receitas com {topico}"

    # O agente retorna a string formatada
    receitas_formatadas = call_agent(buscador, entrada_do_agente_buscador)

    # Extrai os nomes das receitas para uso posterior
    lista_nomes_receitas = extract_recipe_names(receitas_formatadas)

    return receitas_formatadas, lista_nomes_receitas

In [100]:
# --- AGENTE 2: DETALHADOR DE RECEITAS ---
def agente_detalhador_de_receitas(nome_da_receita: str) -> str:
    """
    Detalha uma receita específica usando o nome da receita como input para o Google Search.
    """
    detalhador = Agent(
        name="agente_detalhador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um instrutor de culinária.
        Com base no nome da receita fornecido, sua tarefa é usar a ferramenta de busca do google (Google Search) para buscar detalhes completos sobre essa receita.
        A receita específica a ser detalhada é: "{nome_da_receita}".
        Retorne um texto com a receita detalhando:
        - ingredientes com quantidades (se possível, estimadas)
        - modo de preparo passo a passo
        - tempo de preparo estimado
        - rendimento (porções)
        - dicas culinárias adicionais
        - sugestões de acompanhamentos (se aplicável).
        Sua resposta deve ser clara, organizada e fácil de seguir, como uma receita de um livro de culinária.
        """,
        description="Agente detalhador de receitas no Google",
        tools=[google_search]
    )

    # A entrada para o detalhador é o nome da receita que ele deve buscar
    entrada_do_agente_detalhador = f"Detalhes da receita: {nome_da_receita}"

    receita_detalhada = call_agent(detalhador, entrada_do_agente_detalhador)
    return receita_detalhada

In [101]:
# --- Lógica Principal ---
print("Projeto Receita Culinária")

# --- Obter o Tópico (ingredientes) do Usuário ---
ingredientes_input = input("❓ Que tal preparar uma receita? Liste os ingredientes que você tem disponível, separados por vírgula: ")

if not ingredientes_input.strip():
    print("Você esqueceu de digitar os ingredientes!")
else:
    print(f"Maravilha! Vamos buscar algumas receitas com estes ingredientes: {ingredientes_input}")

    # Chama o agente buscador. Agora ele retorna a string formatada E a lista de nomes.
    receitas_formatadas, lista_nomes_receitas = agente_buscador(ingredientes_input)

    print("\n--- 📝 Resultado do Agente 1 (Buscador de receitas) ---\n")
    if receitas_formatadas.strip(): # Verifica se o agente retornou algo
        display(to_markdown(receitas_formatadas))
    else:
        print("> Nenhuma receita encontrada com os ingredientes fornecidos.")

    # --- Permite ao usuário escolher uma receita se houver resultados ---
    if lista_nomes_receitas:
        while True:
            try:
                escolha_usuario_str = input(
                    "\n❓ Informe o NÚMERO da receita escolhida para detalhar (ou '0' para sair): "
                )
                if escolha_usuario_str == '0':
                    print("Saindo do programa.")
                    break

                escolha_usuario_num = int(escolha_usuario_str)
                # Ajusta para índice base 0
                index_receita_escolhida = escolha_usuario_num - 1

                if 0 <= index_receita_escolhida < len(lista_nomes_receitas):
                    nome_da_receita_escolhida = lista_nomes_receitas[index_receita_escolhida]
                    print(f"Certo, vamos detalhar a receita: {nome_da_receita_escolhida}")

                    # Chama o agente detalhador com o NOME EXATO da receita
                    detalhes_receita = agente_detalhador_de_receitas(nome_da_receita_escolhida)

                    print("\n--- 📝 Resultado do Agente 2 (Detalhador de Receitas) --\n")
                    display(to_markdown(detalhes_receita))
                    print("--------------------------------------------------------------")
                    break # Sai do loop de escolha de receita
                else:
                    print("Número de receita inválido. Por favor, digite um número da lista.")
            except ValueError:
                print("Entrada inválida. Por favor, digite um número.")
    else:
        print("Não há receitas para detalhar.")

print("\nFim do Projeto Receita Culinária.")

Projeto Receita Culinária
❓ Que tal preparar uma receita? Liste os ingredientes que você tem disponível, separados por vírgula: beringela, carne moida, ovos
Maravilha! Vamos buscar algumas receitas com estes ingredientes: beringela, carne moida, ovos

--- 📝 Resultado do Agente 1 (Buscador de receitas) ---



> 1 - Enpanado de carne moída com beringela
> 2 - Berinjela recheada com carne moída
> 3 - Rocambole de Berinjela com Carne Moída
> 4 - Berinjela Recheada com Carne Moída Light
> 5 - Berinjela de forno com carne moída
> 



❓ Informe o NÚMERO da receita escolhida para detalhar (ou '0' para sair): 1
Certo, vamos detalhar a receita: Enpanado de carne moída com beringela

--- 📝 Resultado do Agente 2 (Detalhador de Receitas) --



> Aqui estão os detalhes da receita de "Enpanado de carne moída com beringela", elaborados para serem fáceis de seguir:
> 
>  
> ## Enpanado de Carne Moída com Beringela: Receita Detalhada
> 
> Esta receita combina a versatilidade da carne moída com a textura e sabor únicos da berinjela, resultando em um prato saboroso e nutritivo.
> 
> **Ingredientes:**
> 
> *   1 kg de carne moída
> *   2 berinjelas médias
> *   2 ovos
> *   Farinha de rosca (o suficiente para empanar)
> *   Sal e temperos a gosto (alho, cebola, pimenta do reino, orégano, cheiro verde)
> *   Óleo para fritar
> 
> **Modo de Preparo:**
> 
> 1.  **Preparo da Beringela:**
>     *   Lave bem as berinjelas e corte-as em rodelas de aproximadamente 0,5 cm de espessura.
>     *   Para retirar o amargor, coloque as rodelas de berinjela em água com sal por cerca de 30 minutos. Escorra e seque-as bem com papel toalha.
> 2.  **Tempero da Carne Moída:**
>     *   Em uma tigela, tempere a carne moída com sal, alho picado, cebola picada (ou ralada), pimenta do reino, orégano e cheiro verde a gosto. Misture bem e reserve.
> 3.  **Montagem dos Empanados:**
>     *   Pegue uma rodela de berinjela e coloque uma porção de carne moída temperada por cima, espalhando uniformemente.
>     *   Cubra com outra rodela de berinjela, formando um "sanduíche" de carne moída entre duas fatias de berinjela. Aperte delicadamente para unir.
> 4.  **Empanamento:**
>     *   Em um prato fundo, bata os ovos com um garfo.
>     *   Passe cada "sanduíche" de berinjela na farinha de rosca, certificando-se de cobrir todos os lados.
>     *   Em seguida, mergulhe nos ovos batidos e, novamente, na farinha de rosca para garantir uma camada crocante.
> 5.  **Fritura:**
>     *   Em uma panela funda, aqueça o óleo em fogo médio.
>     *   Frite os empanados de carne moída com berinjela aos poucos, para não abaixar a temperatura do óleo.
>     *   Deixe dourar por igual, virando-os para que fiquem crocantes dos dois lados.
>     *   Retire os empanados com uma escumadeira e coloque-os sobre papel toalha para absorver o excesso de óleo.
> 6.  **Servir:**
>     *   Sirva os empanados de carne moída com berinjela quentes.
> 
> **Tempo de Preparo Estimado:**
> 
> *   Preparo: 30 minutos
> *   Cozimento: 20 minutos
> *   Tempo total: 50 minutos
> 
> **Rendimento:**
> 
> *   Aproximadamente 25 porções, dependendo do tamanho das rodelas de berinjela e da quantidade de carne moída utilizada em cada empanado.
> 
> **Dicas Culinárias Adicionais:**
> 
> *   **Para uma versão mais saudável:** Em vez de fritar, você pode assar os empanados no forno. Coloque-os em uma assadeira untada e leve ao forno pré-aquecido a 200°C por cerca de 30 minutos, virando na metade do tempo para dourar por igual.
> *   **Varie os temperos:** Experimente adicionar outros temperos à carne moída, como cominho, páprica doce ou picante, ou ervas finas.
> *   **Recheio:** Para um toque extra de sabor, adicione queijo mussarela ralado entre as camadas de carne moída e berinjela antes de empanar.
> *   **Molhos:** Sirva com molhos de sua preferência, como molho de tomate caseiro, maionese temperada ou iogurte com ervas.
> 
> **Sugestões de Acompanhamentos:**
> 
> *   Arroz branco
> *   Salada verde
> *   Purê de batata
> *   Legumes cozidos no vapor
> *   Molho de pimenta
> 


--------------------------------------------------------------

Fim do Projeto Receita Culinária.
